In [2]:
import numpy as np
from dmbtools.geometry import utils as gu
from dmbtools.wellbore import survey as srv

In [2]:
np.printoptions(precision=3, suppress=True)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

In [3]:
survey_file_name = r'demo_survey.csv'

In [4]:
survey = np.genfromtxt(survey_file_name, delimiter=',', skip_header=1)
survey

array([[25.00, 0.00, 67.53],
       [100.00, 0.30, 67.53],
       [125.00, 0.21, 64.23],
       ...,
       [16818.00, 91.14, 347.45],
       [16909.00, 89.97, 347.20],
       [17000.00, 89.36, 347.65]])

In [5]:
survey.shape

(441, 3)

In [6]:
def diff_col(col):
    """
    Return a 1D array that is the same length as col that is the
    differance between consecutive elements in col.  The first
    element in the returned array will be zoro.
    """
    col = np.atleast_1d(col)
    x = np.zeros(col.shape[0])
    x[1:] = col[1:] - col[:-1]
    return x

In [7]:
def sum_col(col):
    """
    Return a 1D array that is the same length as col that is the
    differance between consecutive elements in col.  The first
    element in the returned array will be zoro.
    """
    col = np.atleast_1d(col)
    x = np.zeros(col.shape[0])
    x[1:] = col[1:] + col[:-1]
    return x

In [8]:
def delta_md(md):
    return diff_col(md)

In [9]:
def delta_inc(inc_deg):
    return diff_col(inc_deg)

In [10]:
def delta_az(inc_deg):
    """
    IF(ABS(D6-D5)>180,(D6-D5)-360*SIGN(D6-D5),D6-D5))
    """
    diff = diff_col(inc_deg)
    mask = np.abs(diff) > 180.0
    diff[mask] -= 360.0 * np.sign(diff[mask])
    return diff

In [11]:
def build_rate(d_inc, d_md, dog_leg_course_length=100):
    """
    IF(AU$3=2,IF(G6>0,30*F6/G6,0),IF(G6>0,100*F6/G6,0)))
    """
    res = np.zeros(d_md.shape[0])
    mask = d_md > 0.0
    res[mask] = (dog_leg_course_length * d_inc[mask] / d_md[mask])
    return res

In [12]:
def turn_rate(d_az, d_md, dog_leg_course_length=100):
    """
    IF(AU$3=2,IF(G6>0,30*E6/G6,0),IF(G6>0,100*E6/G6,0)))
    """
    res = np.zeros(d_md.shape[0])
    mask = d_md > 0.0
    res[mask] = (dog_leg_course_length * d_az[mask] / d_md[mask])
    return res

In [13]:
def effective_turn_rate(inc_deg, tr):
    """
    I6*SIN(0.5*(C6+C5)*0.0174533))
    This is lateral dogleg severity
    """
    rad_per_180 = np.pi / 180.0
    res = tr * np.sin( 0.5 * sum_col(inc_deg) * rad_per_180)
    return res

In [14]:
def delta_br(br):
    return diff_col(br)

In [15]:
def delta_etr(etr):
    return diff_col(etr)

In [16]:
def unwanted_vertical_curvature(d_md, d_br, dog_leg_course_length=100):
    """
    M5+0.5*ABS(K6*G6/100)
    """
    res = 0.5 * np.abs(d_br * d_md / dog_leg_course_length)
    return np.cumsum(res)

In [17]:
def unwanted_lateral_curvature(d_md, d_etr, dog_leg_course_length=100):
    """
    N5+0.5*ABS(L6*G6/100)
    """
    res = 0.5 * np.abs(d_etr * d_md / dog_leg_course_length)
    return np.cumsum(res)

In [18]:
def total_vertical_curvature(d_md, br, dog_leg_course_length=100):
    """
    O6+ABS(H7*G7)/100)
    """
    res = np.abs(d_md * br / dog_leg_course_length)
    return np.cumsum(res)

In [19]:
def total_lateral_curvature(d_md, etr, dog_leg_course_length=100):
    """
    X P5+ABS(E6*SIN(0.5*(C5+C6)*0.0174533))
    P6+ABS(J7*G7)/100)
    """
    res = np.abs(d_md * etr / dog_leg_course_length)
    return np.cumsum(res)

In [20]:
def vertical_tortuosity_index(uvc, tvc):
    """
    IF(ABS(O6-M6)>0.5,(M6/(O6-M6)),1)
    """
    res = np.ones(uvc.shape[0])
    diff = tvc - uvc
    mask = np.abs(diff) > 0.5
    res[mask] = uvc[mask] / diff[mask]
    return res

In [21]:
def lateral_tortuosity_index(ulc, tlc):
    """
    IF(ABS(P6-N6)>0.5,(N6/(P6-N6)),1)
    """
    res = np.ones(ulc.shape[0])
    diff = tlc - ulc
    mask = np.abs(diff) > 0.5
    res[mask] = ulc[mask] / diff[mask]
    return res

In [22]:
def dogleg_severity(br, etr):
    """
    SQRT(H6*H6+J6*J6)
    """
    return np.sqrt(br * br + etr * etr)

In [23]:
def dogleg_severity_difference(dls):
    """
    ABS(S6-S5)
    """
    return np.abs(diff_col(dls))

In [24]:
def cummulative_curvature(d_md, dls, dog_leg_course_length=100):
    """
    U5+S6*(B6-B5)/100)
    """
    return np.cumsum((dls * d_md) / dog_leg_course_length)

In [25]:
def unwanted_curvature(d_md, dls_diff, dog_leg_course_length=100):
    """
    V5+T6*(B6-B5)*0.5/100)
    NOTE: this is UC in the plane of curvature, not vertical and lateral
    """
    return np.cumsum((dls_diff * d_md * 0.5) / dog_leg_course_length)

In [26]:
def tortuosity_index(vti, lti):
    """
    SQRT(Q6^2+R6^2)
    """
    return np.sqrt(vti * vti + lti * lti)

In [27]:
def tortuosity_index_2(tvc, tlc, normalising_curvature=90):
    """
    Per Angus pptx CSPG talk 01-27-21
    """
    return np.sqrt(tvc * tvc + tlc * tlc) / normalising_curvature

In [28]:
def toolface_drilled(br, etr):
    """
    ATAN2(H7,J7+0.00001)/0.0174533)
    """
    rad_per_180 = np.pi / 180.0
    return np.arctan2(etr, br) / rad_per_180

In [29]:
rpt = 10

In [30]:
md = survey[:,0]
inc = survey[:,1]
az = survey[:,2]

In [31]:
d_md = diff_col(md)
d_md[:rpt]

array([0.00, 75.00, 25.00, 25.00, 25.00, 25.00, 25.00, 25.00, 25.00,
       25.00])

In [32]:
d_inc = delta_inc(inc)
d_inc[:rpt]

array([0.00, 0.30, -0.09, 0.05, 0.00, 0.05, 0.01, 0.00, 0.02, 0.04])

In [33]:
d_az = delta_az(az)
d_az[:rpt]

array([0.00, 0.00, -3.30, 18.35, 3.88, -0.60, 7.32, -5.70, -3.30, -1.86])

In [34]:
br = build_rate(d_inc, d_md)
br[:rpt]

array([0.00, 0.40, -0.36, 0.20, 0.00, 0.20, 0.04, 0.00, 0.08, 0.16])

In [35]:
tr = turn_rate(d_az, d_md)
tr[:rpt]

array([0.00, 0.00, -13.20, 73.40, 15.52, -2.40, 29.28, -22.80, -13.20,
       -7.44])

In [36]:
etr = effective_turn_rate(inc, tr)
etr[:rpt]

array([0.00, 0.00, -0.06, 0.30, 0.07, -0.01, 0.16, -0.13, -0.08, -0.05])

In [37]:
d_br = delta_br(br)
d_br[:rpt]

array([0.00, 0.40, -0.76, 0.56, -0.20, 0.20, -0.16, -0.04, 0.08, 0.08])

In [38]:
d_etr = delta_br(etr)
d_etr[:rpt]

array([0.00, 0.00, -0.06, 0.36, -0.23, -0.08, 0.17, -0.29, 0.05, 0.03])

In [39]:
uvc = unwanted_vertical_curvature(d_md, d_br)
uvc[:rpt]

array([0.00, 0.15, 0.24, 0.32, 0.34, 0.36, 0.39, 0.39, 0.40, 0.41])

In [40]:
ulc = unwanted_lateral_curvature(d_md, d_etr)
ulc[:rpt]

array([0.00, 0.00, 0.01, 0.05, 0.08, 0.09, 0.11, 0.15, 0.16, 0.16])

In [41]:
tvc = total_vertical_curvature(d_md, br)
tvc[:rpt]

array([0.00, 0.30, 0.39, 0.44, 0.44, 0.49, 0.50, 0.50, 0.52, 0.56])

In [42]:
tlc = total_lateral_curvature(d_md, etr)
tlc[:rpt]

array([0.00, 0.00, 0.01, 0.09, 0.11, 0.11, 0.15, 0.18, 0.20, 0.21])

In [43]:
vti = vertical_tortuosity_index(uvc, tvc)[:rpt]
vti[:rpt]

array([1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00])

In [44]:
lti = lateral_tortuosity_index(ulc, tlc)[:rpt]
lti[:rpt]

array([1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00])

In [45]:
dls = dogleg_severity(br, etr)
dls[:rpt]

array([0.00, 0.40, 0.36, 0.36, 0.07, 0.20, 0.17, 0.13, 0.11, 0.17])

In [46]:
dls_diff = dogleg_severity_difference(dls)
dls_diff[:rpt]

array([0.00, 0.40, 0.04, 0.00, 0.29, 0.13, 0.03, 0.04, 0.02, 0.06])

In [47]:
cumm_curv = cummulative_curvature(d_md, dls, dog_leg_course_length=100)
cumm_curv[:rpt]

array([0.00, 0.30, 0.39, 0.48, 0.50, 0.55, 0.59, 0.62, 0.65, 0.69])

In [48]:
u_crv = unwanted_curvature(d_md, dls_diff, dog_leg_course_length=100)
u_crv[:rpt]

array([0.00, 0.15, 0.15, 0.15, 0.19, 0.21, 0.21, 0.22, 0.22, 0.23])

In [49]:
ti = tortuosity_index(vti, lti)
ti[:rpt]

array([1.41, 1.41, 1.41, 1.41, 1.41, 1.41, 1.41, 1.41, 1.41, 1.41])

In [50]:
ti2 = tortuosity_index_2(tvc, tlc, normalising_curvature=90)
ti2[:rpt]

array([0.00, 0.00, 0.00, 0.00, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

In [51]:
tface_drill = toolface_drilled(br, etr)
print(len(tface_drill))
tface_drill[:rpt]

441


array([0.00, 0.00, -170.73, 56.40, 90.00, -3.42, 76.05, -90.00, -43.54,
       -16.29])

### Work arc2chord

In [52]:
np.printoptions(precision=8, suppress=True)
np.set_printoptions(formatter={'float': lambda x: "{0:0.8f}".format(x)})

In [53]:
tx = gu.toUnitDir(survey[:, 1], survey[:, 2])
tx

array([[0.00000000, 0.00000000, 1.00000000],
       [0.00200118, 0.00483845, 0.99998629],
       [0.00159347, 0.00330067, 0.99999328],
       ...,
       [0.97591355, -0.21724850, -0.01989544],
       [0.97514922, -0.22154847, 0.00052360],
       [0.97679836, -0.21386960, 0.01116987]])

In [54]:
ang = gu.angle(tx[:-1], tx[1:])

In [55]:
ang

array([0.00523599, 0.00159093, 0.00156620, 0.00030724, 0.00087421,
       0.00072319, 0.00055539, 0.00048141, 0.00072723, 0.00076676,
       0.00163171, 0.00087257, 0.00069838, 0.00045495, 0.00134567,
       0.00200013, 0.00100395, 0.00154441, 0.00133598, 0.00263006,
       0.00171992, 0.00067242, 0.00073130, 0.00091246, 0.00200993,
       0.00026245, 0.00086411, 0.00011697, 0.00032535, 0.00077186,
       0.00070027, 0.00069814, 0.00103868, 0.00074038, 0.00021347,
       0.00057830, 0.00087740, 0.00048514, 0.00165571, 0.00099006,
       0.00109369, 0.00094951, 0.00035905, 0.00036940, 0.00099404,
       0.00140126, 0.00170250, 0.00115429, 0.00230075, 0.00196399,
       0.00213259, 0.00038482, 0.00095713, 0.00077178, 0.00065328,
       0.00110047, 0.00110835, 0.00122197, 0.00264581, 0.00052372,
       0.00245771, 0.00204029, 0.00061423, 0.00058134, 0.00192410,
       0.00133656, 0.00109649, 0.00230379, 0.00342335, 0.00296910,
       0.00092812, 0.00234732, 0.00053743, 0.00036019, 0.00065

In [56]:
R = d_md[1:] / ang
R

array([14323.94487827, 15714.12114610, 15962.19864563, 81370.06732058,
       28597.38204644, 34568.88027888, 45013.39805990, 51930.51342603,
       34377.13962355, 32604.77284928, 15321.30927620, 28651.14747953,
       35797.16634551, 54951.36509892, 18578.09117502, 12499.17514280,
       24901.60497879, 16187.37964211, 18712.91817563, 9505.48013383,
       14535.56052557, 37179.18756784, 34185.88288112, 27398.43234475,
       12438.23922446, 95255.70589710, 28931.58304025, 213729.38905916,
       76840.09493053, 32389.20866696, 35700.28509428, 35809.40407117,
       24068.90737013, 33766.35152942, 117113.78294380, 43230.42897972,
       28493.29309515, 51531.72953271, 15099.27500995, 25251.08885680,
       22858.32935236, 26329.23485618, 69627.30515381, 67676.97429220,
       25149.95771938, 17841.14724449, 14684.25747069, 21658.28887135,
       10866.01262304, 12729.17741639, 11722.84828572, 64964.79874686,
       26119.85814159, 32392.49130463, 38268.28889246, 22717.48491956,
     

In [57]:
a2c_coor = R * (1.0 - np.cos(ang / 2.0))
a2c_coor

array([0.04908736, 0.00497164, 0.00489438, 0.00096012, 0.00273189,
       0.00225998, 0.00173559, 0.00150441, 0.00227259, 0.00239612,
       0.00509911, 0.00272677, 0.00218244, 0.00142171, 0.00420522,
       0.00625041, 0.00313735, 0.00482629, 0.00417492, 0.00821894,
       0.00537475, 0.00210131, 0.00228530, 0.00285144, 0.00628103,
       0.00082016, 0.00270034, 0.00036553, 0.00101672, 0.00241207,
       0.00218836, 0.00218169, 0.00324589, 0.00231369, 0.00066709,
       0.00180718, 0.00274187, 0.00151606, 0.00517409, 0.00309393,
       0.00341779, 0.00296723, 0.00112205, 0.00115438, 0.00310637,
       0.00437892, 0.00532032, 0.00360716, 0.00718985, 0.00613747,
       0.00666434, 0.00120257, 0.00299102, 0.00241182, 0.00204151,
       0.00343898, 0.00346361, 0.00381865, 0.00826815, 0.00163662,
       0.00768033, 0.00637589, 0.00191948, 0.00181667, 0.00601281,
       0.00417676, 0.00342653, 0.00719935, 0.01069795, 0.00927843,
       0.00290038, 0.00733538, 0.00167947, 0.00112559, 0.00203

In [58]:
chords = srv.arc2chord(tx[:-1], tx[1:], d_md[1:])
chords

(array([[0.07504455, 0.18144223, 74.99965731],
        [0.04493322, 0.10173898, 24.99974997],
        [0.02724374, 0.09750639, 24.99979245],
        ...,
        [88.95228972, -19.16715875, -0.10323174],
        [88.77658191, -19.96598754, -0.88145084],
        [88.81491023, -19.81181081, 0.53206081]]),
 array([0.00523599, 0.00159093, 0.00156620, 0.00030724, 0.00087421,
        0.00072319, 0.00055539, 0.00048141, 0.00072723, 0.00076676,
        0.00163171, 0.00087257, 0.00069838, 0.00045495, 0.00134567,
        0.00200013, 0.00100395, 0.00154441, 0.00133598, 0.00263006,
        0.00171992, 0.00067242, 0.00073130, 0.00091246, 0.00200993,
        0.00026245, 0.00086411, 0.00011697, 0.00032535, 0.00077186,
        0.00070027, 0.00069814, 0.00103868, 0.00074038, 0.00021347,
        0.00057830, 0.00087740, 0.00048514, 0.00165571, 0.00099006,
        0.00109369, 0.00094951, 0.00035905, 0.00036940, 0.00099404,
        0.00140126, 0.00170250, 0.00115429, 0.00230075, 0.00196399,
        0.00213

In [59]:
chord_lens = np.linalg.norm(chords[0], axis=1) / 2.0
chord_lens

array([37.49995716, 12.49999868, 12.49999872, 12.49999995, 12.49999960,
       12.49999973, 12.49999984, 12.49999988, 12.49999972, 12.49999969,
       12.49999861, 12.49999960, 12.49999975, 12.49999989, 12.49999906,
       12.49999792, 12.49999948, 12.49999876, 12.49999907, 12.49999640,
       12.49999846, 12.49999976, 12.49999972, 12.49999957, 12.49999790,
       12.49999996, 12.49999961, 12.49999999, 12.49999994, 12.49999969,
       12.49999974, 12.49999975, 12.49999944, 12.49999971, 12.49999998,
       12.49999983, 12.49999960, 12.49999988, 12.49999857, 12.49999949,
       12.49999938, 12.49999953, 12.49999993, 12.49999993, 12.49999949,
       12.49999898, 12.49999849, 12.49999931, 12.49999724, 12.49999799,
       12.49999763, 12.49999992, 12.49999952, 12.49999969, 12.49999978,
       12.49999937, 12.49999936, 12.49999922, 12.49999635, 12.49999986,
       12.49999685, 12.49999783, 12.49999980, 12.49999982, 12.49999807,
       12.49999907, 12.49999937, 12.49999724, 12.49999390, 12.49

In [60]:
np.tan(0.25 * chords[1]) * (0.5 * np.linalg.norm(chords[0], axis=1))

array([0.04908736, 0.00497164, 0.00489438, 0.00096012, 0.00273189,
       0.00225998, 0.00173559, 0.00150441, 0.00227259, 0.00239612,
       0.00509911, 0.00272677, 0.00218244, 0.00142171, 0.00420522,
       0.00625041, 0.00313735, 0.00482629, 0.00417492, 0.00821894,
       0.00537475, 0.00210131, 0.00228530, 0.00285144, 0.00628103,
       0.00082016, 0.00270034, 0.00036553, 0.00101672, 0.00241207,
       0.00218836, 0.00218169, 0.00324589, 0.00231369, 0.00066709,
       0.00180718, 0.00274187, 0.00151606, 0.00517409, 0.00309393,
       0.00341779, 0.00296723, 0.00112205, 0.00115438, 0.00310637,
       0.00437892, 0.00532032, 0.00360716, 0.00718985, 0.00613747,
       0.00666434, 0.00120257, 0.00299102, 0.00241182, 0.00204151,
       0.00343898, 0.00346361, 0.00381865, 0.00826815, 0.00163662,
       0.00768033, 0.00637589, 0.00191948, 0.00181667, 0.00601281,
       0.00417676, 0.00342653, 0.00719935, 0.01069795, 0.00927843,
       0.00290038, 0.00733538, 0.00167947, 0.00112559, 0.00203

In [61]:
R * (1.0 - np.cos(ang / 2.0))

array([0.04908736, 0.00497164, 0.00489438, 0.00096012, 0.00273189,
       0.00225998, 0.00173559, 0.00150441, 0.00227259, 0.00239612,
       0.00509911, 0.00272677, 0.00218244, 0.00142171, 0.00420522,
       0.00625041, 0.00313735, 0.00482629, 0.00417492, 0.00821894,
       0.00537475, 0.00210131, 0.00228530, 0.00285144, 0.00628103,
       0.00082016, 0.00270034, 0.00036553, 0.00101672, 0.00241207,
       0.00218836, 0.00218169, 0.00324589, 0.00231369, 0.00066709,
       0.00180718, 0.00274187, 0.00151606, 0.00517409, 0.00309393,
       0.00341779, 0.00296723, 0.00112205, 0.00115438, 0.00310637,
       0.00437892, 0.00532032, 0.00360716, 0.00718985, 0.00613747,
       0.00666434, 0.00120257, 0.00299102, 0.00241182, 0.00204151,
       0.00343898, 0.00346361, 0.00381865, 0.00826815, 0.00163662,
       0.00768033, 0.00637589, 0.00191948, 0.00181667, 0.00601281,
       0.00417676, 0.00342653, 0.00719935, 0.01069795, 0.00927843,
       0.00290038, 0.00733538, 0.00167947, 0.00112559, 0.00203

In [5]:
aaa = np.array([1,2,3,4,5])

In [24]:
xxx = np.column_stack([aaa, aaa])
xxx

array([[1, 1],
       [2, 2],
       [3, 3],
       [4, 4],
       [5, 5]])

In [9]:
cols = ['aaa',
        'bbb',
        'ccc']

In [14]:
dict(zip(cols, range(len(cols))))

{'aaa': 0, 'bbb': 1, 'ccc': 2}

In [23]:
cols[-2:]

['bbb', 'ccc']

In [25]:
xxx[-3:]

array([[3, 3],
       [4, 4],
       [5, 5]])